In [23]:
import zipfile
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split
import shutil

In [24]:
!git clone https://github.com/avaneeshp23/AVHackathon.git

fatal: destination path 'AVHackathon' already exists and is not an empty directory.


In [25]:
train_df = pd.read_csv('/content/AVHackathon/train/train.csv')
train_df.head()
train_df.shape

(1646, 2)

In [26]:
train_df.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [27]:
test_df = pd.read_csv('/content/AVHackathon/test.csv')
test_df.shape

(706, 2)

code to delete folders

In [ ]:
# folder_path = '/content/train_images/non_emergency'

# # Check if the folder exists
# if os.path.exists(folder_path):
#     # Iterate over all the files in the folder
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)
#         try:
#             # Check if it's a file and delete it
#             if os.path.isfile(file_path) or os.path.islink(file_path):
#                 os.unlink(file_path)
#             # Check if it's a directory and delete it
#             elif os.path.isdir(file_path):
#                 shutil.rmtree(file_path)
#         except Exception as e:
#             print(f'Failed to delete {file_path}. Reason: {e}')
# else:
#     print(f'The folder {folder_path} does not exist.')

In [28]:
os.makedirs('train_images', exist_ok=True)
os.makedirs('test_images', exist_ok=True)

In [29]:
def copy_images(df, src_folder, dest_folder):
    for index, row in df.iterrows():
        image_name = row['image_names']
        src_path = os.path.join(src_folder, image_name)
        dest_path = os.path.join(dest_folder, image_name)
        shutil.copy(src_path, dest_path)

In [ ]:
#move_images(train_df, '/content/AVHackathon/train/images', 'train_images')

In [30]:
os.makedirs('train_images/1', exist_ok=True)
os.makedirs('train_images/0', exist_ok=True)

for index, row in train_df.iterrows():
    image_name = row['image_names']
    emergency_or_not = row['emergency_or_not']

    # Define source and destination paths
    src_path = os.path.join('/content/AVHackathon/train/images', image_name)
    if emergency_or_not == 1:
        dest_path = os.path.join('train_images/1', image_name)
    else:
        dest_path = os.path.join('train_images/0', image_name)

    # Copy the image to the respective folder
    shutil.copy(src_path, dest_path)

In [31]:
os.makedirs('test_images', exist_ok=True)
copy_images(test_df, '/content/AVHackathon/train/images', 'test_images')

In [32]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean/std
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is available


In [34]:
checkpoints_dir = 'train_images/.ipynb_checkpoints'
if os.path.exists(checkpoints_dir):
    shutil.rmtree(checkpoints_dir)
train_dataset = datasets.ImageFolder(root='train_images', transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet18

# Define a simple CNN model (using ResNet18 as a starting point)
model = resnet18(pretrained=True)  # Using pre-trained ResNet18
num_ftrs = model.fc.in_features  # The number of input features to the final fully connected layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 2)
)  # We have 2 classes, so change the final layer

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Data transformations without augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
dataset = datasets.ImageFolder(root='/content/train_images', transform=transform)

# Create DataLoader
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
epochs = 12

for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # Step the scheduler
    scheduler.step()

Epoch 1/12, Loss: 0.3528, Accuracy: 86.15%
Epoch 2/12, Loss: 0.1986, Accuracy: 92.77%
Epoch 3/12, Loss: 0.1092, Accuracy: 96.17%
Epoch 4/12, Loss: 0.0719, Accuracy: 97.51%
Epoch 5/12, Loss: 0.0690, Accuracy: 97.39%
Epoch 6/12, Loss: 0.0341, Accuracy: 98.97%
Epoch 7/12, Loss: 0.0140, Accuracy: 99.57%
Epoch 8/12, Loss: 0.0103, Accuracy: 99.64%
Epoch 9/12, Loss: 0.0080, Accuracy: 99.76%
Epoch 10/12, Loss: 0.0077, Accuracy: 99.70%
Epoch 11/12, Loss: 0.0061, Accuracy: 99.82%
Epoch 12/12, Loss: 0.0055, Accuracy: 99.82%


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# import torchvision
# from torchvision import datasets, transforms
# from torchvision.models import resnet18

# # Define a simple CNN model (using ResNet18 as a starting point)
# model = resnet18(pretrained=True)  # Using pre-trained ResNet18
# num_ftrs = model.fc.in_features  # The number of input features to the final fully connected layer
# model.fc = nn.Linear(num_ftrs, 2)  # We have 2 classes, so change the final layer

# # Move the model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()  # For multi-class classification
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Define the learning rate scheduler
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# # Training loop
# epochs = 10

# for epoch in range(epochs):
#     model.train()  # Set model to training mode
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()  # Zero the parameter gradients

#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         # Calculate accuracy
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     epoch_loss = running_loss / len(train_loader)
#     epoch_acc = 100 * correct / total
#     print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

#     # Step the scheduler
#     scheduler.step()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.4530, Accuracy: 81.83%
Epoch 2/10, Loss: 0.2011, Accuracy: 91.86%
Epoch 3/10, Loss: 0.1496, Accuracy: 93.92%
Epoch 4/10, Loss: 0.0893, Accuracy: 96.96%
Epoch 5/10, Loss: 0.0830, Accuracy: 96.96%
Epoch 6/10, Loss: 0.0397, Accuracy: 98.48%
Epoch 7/10, Loss: 0.0192, Accuracy: 99.39%
Epoch 8/10, Loss: 0.0147, Accuracy: 99.57%
Epoch 9/10, Loss: 0.0080, Accuracy: 99.70%
Epoch 10/10, Loss: 0.0082, Accuracy: 99.76%


In [ ]:
# epochs = 10

# for epoch in range(epochs):
#     model.train()  # Set model to training mode
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()  # Zero the parameter gradients

#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         # Calculate accuracy
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     epoch_loss = running_loss / len(train_loader)
#     epoch_acc = 100 * correct / total
#     print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

Epoch 1/10, Loss: 0.4246, Accuracy: 83.96%
Epoch 2/10, Loss: 0.1896, Accuracy: 92.41%
Epoch 3/10, Loss: 0.1161, Accuracy: 96.66%
Epoch 4/10, Loss: 0.0728, Accuracy: 97.21%
Epoch 5/10, Loss: 0.1018, Accuracy: 96.11%
Epoch 6/10, Loss: 0.0755, Accuracy: 97.02%
Epoch 7/10, Loss: 0.0636, Accuracy: 97.75%
Epoch 8/10, Loss: 0.0405, Accuracy: 98.72%
Epoch 9/10, Loss: 0.0346, Accuracy: 98.60%
Epoch 10/10, Loss: 0.0758, Accuracy: 96.96%


In [40]:
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

test_images_folder = '/content/test_images'
result_df = pd.DataFrame(columns=['image_names', 'emergency_or_not'])

for image_name in os.listdir(test_images_folder):
    image_path = os.path.join(test_images_folder, image_name)

    # Open the image
    image = Image.open(image_path).convert('RGB')

    # Apply transformations
    image = test_transforms(image)

    # Add batch dimension
    image = image.unsqueeze(0).to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    # Append the result to the DataFrame
    result_df = pd.concat([result_df, pd.DataFrame({'image_names': [image_name], 'emergency_or_not': [predicted.item()]})], ignore_index=True)



In [41]:
test_csv_path = '/content/AVHackathon/test.csv'
test_df = pd.read_csv(test_csv_path)

In [42]:
if 'emergency_or_not' not in test_df.columns:
    test_df['emergency_or_not'] = None

In [43]:
result_df.head()

,image_names,emergency_or_not
0,849.jpg,1
1,1747.jpg,0
2,324.jpg,1
3,1817.jpg,0
4,291.jpg,1


In [44]:
for index, row in result_df.iterrows():
    test_df.loc[test_df['image_names'] == row['image_names'], 'emergency_or_not'] = row['emergency_or_not']

In [45]:
test_df.to_csv(test_csv_path, index=False)

In [46]:
test_df.head()

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,1
2,2082.jpg,0
3,808.jpg,1
4,1907.jpg,0


In [47]:
# prompt: updated_test_df  = pd.read_csv(test_csv_path)

updated_test_df  = pd.read_csv(test_csv_path)
updated_test_df.head()

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,1
2,2082.jpg,0
3,808.jpg,1
4,1907.jpg,0
